# Pregunta 7
Realizar una unión interna en dos DataFrames <br>
Con dos DataFrames: <br>
empleado_df y departamento_df <br>
Debe unirlos en dept_id para determinar a qué departamento pertenece cada empleado

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [6]:
spark = SparkSession.builder.appName('Pregunta7').getOrCreate()

df = spark.read.csv('./datasets/employees.csv', header=True, inferSchema=True)


# Creamos df con Department y dept_id -> Sales:1, HR:2, IT:3
df_join1 = df.withColumn('dept_id',
                              F.when(F.col('Department') == 'Sales', 1)
                              .when(F.col('Department') == 'HR', 2)
                              .when(F.col('Department') == 'IT', 3))\
                                .drop('Department')

# Creamos df solo con dept_id y Department
df_join2 = df.select('Department').distinct()\
                .withColumn('dept_id',
                                F.when(F.col('Department') == 'Sales', 1)
                                .when(F.col('Department') == 'HR', 2)
                                .when(F.col('Department') == 'IT', 3))


In [8]:
final_join = df_join1.join(df_join2, on='dept_id', how='inner')
final_join.show()

+-------+---+------------------+---+------+------+------------+-----------------+----------+--------+-----------+-------+----------+
|dept_id| ID|              Name|Age|Gender|Salary|Joining Date|Performance Score|Experience|  Status|   Location|Session|Department|
+-------+---+------------------+---+------+------+------------+-----------------+----------+--------+-----------+-------+----------+
|      2|  1|      Cory Escobar| 48|Female|  5641|  2015-05-03|              2.0|        16|  Active|   New York|  Night|        HR|
|      1|  2|   Timothy Sanchez| 25| Other|  4249|  2020-11-09|              2.0|        11|Inactive|Los Angeles|Evening|     Sales|
|      1|  3|      Chad Nichols| 57| Other|  3058|  2019-02-12|             NULL|         1|Inactive|   New York|Morning|     Sales|
|      3|  4|Christine Williams| 58|Female|  5895|  2017-09-08|              2.0|        13|Inactive|Los Angeles|Evening|        IT|
|      3|  5|      Amber Harris| 35| Other|  4317|  2020-02-15|      